<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2025-04-17 23:20:42--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  60.5MB/s    in 4.2s    

2025-04-17 23:20:46 (59.4 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [2]:
import zipfile
with zipfile.ZipFile('concrete_data_week4.zip', 'r') as zip_ref:
    zip_ref.extractall()

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [3]:
import keras
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense


In [4]:
batch_size = 100
image_resize = (224, 224)
num_classes = 2

In [5]:
# prepare the data
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=image_resize,
    batch_size=batch_size,
    class_mode='categorical',
)

validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=image_resize,
    batch_size=batch_size,
    class_mode='categorical',
)

Found 30001 images belonging to 2 classes.
Found 9501 images belonging to 2 classes.


In [6]:
# build the model
model = Sequential()

model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',))

model.add(Dense(num_classes, activation='softmax'))

#freeze the pre-trained parameters
model.layers[0].trainable = False



58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [23]:
#print model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 512)            │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │         1,026 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,717,768 (56.14 MB)

 Trainable params: 1,026 (4.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

 Optimizer params: 2,054 (8.03 KB)

In [7]:
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
# train the model
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2
301/301 ━━━━━━━━━━━━━━━━━━━━ 235s 680ms/step - accuracy: 0.9356 - loss: 0.1758 - val_accuracy: 0.9944 - val_loss: 0.0221
Epoch 2/2
301/301 ━━━━━━━━━━━━━━━━━━━━ 235s 781ms/step - accuracy: 0.9956 - loss: 0.0196 - val_accuracy: 0.9968 - val_loss: 0.0133


<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model.
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [ ]:
# load saved resenet50 model
model_resnet50 = keras.models.load_model('classifier_resnet_model.h5')

In [12]:
# prepare test data
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=image_resize,
    shuffle=False,
)

Found 500 images belonging to 2 classes.


In [13]:
# evaluate models on test data
steps_per_epoch_test = len(test_generator)
evaluation_resnet50 = model_resnet50.evaluate(test_generator, steps=steps_per_epoch_test)
evaluation_vgg16 = model.evaluate(test_generator, steps=steps_per_epoch_test)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


16/16 ━━━━━━━━━━━━━━━━━━━━ 11s 264ms/step - accuracy: 0.9995 - loss: 0.0045
16/16 ━━━━━━━━━━━━━━━━━━━━ 21s 680ms/step - accuracy: 0.9990 - loss: 0.0104


In [14]:
# print performances of classifiers
print('ResNet50 Model Performance')
print(f'Loss: {evaluation_resnet50[0]}, Accuracy: {evaluation_resnet50[1]}')
print('-'*50)
print('VGG16 Model Peformance')
print(f'Loss: {evaluation_vgg16[0]}, Accuracy: {evaluation_vgg16[1]}')

ResNet50 Model Performance
Loss: 0.006216392386704683, Accuracy: 0.9980000257492065
--------------------------------------------------
VGG16 Model Peformance
Loss: 0.010732412338256836, Accuracy: 0.9980000257492065


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li>
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [15]:
import numpy as np
# predict class of the images in the test data
predictions_resnet50 = model_resnet50.predict(test_generator)
predictions_vgg16 = model.predict(test_generator)
predictions_vgg16.shape

16/16 ━━━━━━━━━━━━━━━━━━━━ 8s 325ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 4s 187ms/step


(500, 2)

In [22]:
# show actual classes of first five images in the test set
print('-'*50)
print('Actual Classes')
print('-'*50)
_ = [print('Positive') if pred == 1 else print('Negative') for pred in test_generator.classes[:5]]

# report class prediction of first five images in the test set
print('-'*50)
print('ResNet50 Model Predictions')
print('-'*50)
predictions_resnet50_classes = np.argmax(predictions_resnet50[:5], axis=1)
_ = [print('Positive') if pred == 1 else print('Negative') for pred in predictions_resnet50_classes]
print('-'*50)
print('VGG16 Model Predictions')
print('-'*50)
predictions_vgg16_classes = np.argmax(predictions_vgg16[:5], axis=1)
_ = [print('Positive') if pred == 1 else print('Negative') for pred in predictions_vgg16_classes]



--------------------------------------------------
Actual Classes
--------------------------------------------------
Negative
Negative
Negative
Negative
Negative
--------------------------------------------------
ResNet50 Model Predictions
--------------------------------------------------
Negative
Negative
Negative
Negative
Negative
--------------------------------------------------
VGG16 Model Predictions
--------------------------------------------------
Negative
Negative
Negative
Negative
Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).